# Examining problematic sulfamide geometries

In [1]:
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.units import unit

from openff.interchange import Interchange
from openff.interchange.drivers import get_gromacs_energies, get_openmm_energies
import multiprocessing
import tqdm
import numpy as np
import sys
import openmm
from openff.units.openmm import from_openmm
import os
# suppress stereochemistry warnings
import logging
from rdkit import Chem
import itertools

from openff.qcsubmit.results import TorsionDriveResultCollection, OptimizationResultCollection
import logging
logging.getLogger("openff").setLevel(logging.ERROR)

from openff.qcsubmit.results.filters import SMARTSFilter

In [2]:
def minimize_energy(mol,ff):
    topo = mol.to_topology()
    interchange = Interchange.from_smirnoff(force_field=ff, topology=topo,allow_nonintegral_charges=True)#,charge_from_molecules=[mol]) #,charge_from_molecules=[mol])
    integrator = openmm.VerletIntegrator(1 * openmm.unit.femtoseconds)
    simulation = interchange.to_openmm_simulation(integrator)

    # We'll store energies in two lists
    initial_energies = []
    minimized_energies = []

    # And minimized conformers in a second molecule
    minimized_molecule = Molecule.from_topology(topo)
    minimized_molecule.conformers.clear()

    conformer = mol.conformers[0]
    # Tell the OpenMM Simulation the positions of this conformer
    simulation.context.setPositions(conformer.to_openmm())

    # Keep a record of the initial energy
    initial_energies.append(
        simulation.context.getState(getEnergy=True).getPotentialEnergy()
    )

    # Perform the minimization
    simulation.minimizeEnergy()#tolerance=Quantity(value=0.0026255, unit=kilojoule/mole))#tolerance=5e-9)

    # Record minimized energy and positions
    min_state = simulation.context.getState(getEnergy=True, getPositions=True)

    minimized_energies.append(min_state.getPotentialEnergy())
    minimized_molecule.add_conformer(from_openmm(min_state.getPositions()))
    return initial_energies,minimized_energies,minimized_molecule


# Finding test molecules from benchmarking dataset
Filter for N-S(=O)(=O)-N

In [27]:
industry_benchmark = OptimizationResultCollection.parse_file('../../datasets/filtered-industry.json')

In [28]:
industry_benchmark_filter = industry_benchmark.filter(SMARTSFilter(smarts_to_include=['[#7]~[#16X4:1](=[#8])(~[#7])=[#8]']))

In [29]:
industry_benchmark_filter.n_results

127

In [30]:
industry_benchmark_rm = industry_benchmark_filter.to_records()

Choosing a few problem cases:

# Molecule 19, QCArchive ID 36971837
Bad with Sage 2.0.0, fixed in 2.1 and 2.2

Has many conformers--index 19-28. Some are distorted, some aren't

In [58]:
test_mol1 = industry_benchmark_rm[19][1]
test_mol1

NGLWidget()

In [59]:
test_mol1.to_file('qcaid_36971837.sdf',file_format='sdf')

In [3]:
test_mol1=Molecule.from_file('qcaid_36971837.sdf')

In [4]:
ie_sage200_qm1,me_sage200_qm1,min_mol_sage200_qm1 = minimize_energy(test_mol1,ForceField('openff_unconstrained-2.0.0.offxml'))

In [5]:
ie_sage210_qm1,me_sage210_qm1,min_mol_sage210_qm1 = minimize_energy(test_mol1,ForceField('openff_unconstrained-2.1.0.offxml'))

In [6]:
min_mol_sage210_qm1

NGLWidget()

In [7]:
ie_sage220_qm1,me_sage220_qm1,min_mol_sage220_qm1 = minimize_energy(test_mol1,ForceField('../../../openff_unconstrained-2.2.0-rc1.offxml',allow_cosmetic_attributes=True))

In [8]:
min_mol_sage220_qm1.to_file('qcaid_36971837_optsage220.sdf',file_format='sdf')

In [9]:
opt1_sage200 = min_mol_sage200_qm1.to_rdkit()
opt1_sage200_conf = opt1_sage200.GetConformer()

In [10]:
opt1_sage210 = min_mol_sage210_qm1.to_rdkit()
opt1_sage210_conf = opt1_sage210.GetConformer()

In [12]:
opt1_sage220 = min_mol_sage220_qm1.to_rdkit()
opt1_sage220_conf = opt1_sage220.GetConformer()

In [13]:
qm_rdkit1 = test_mol1.to_rdkit()
qm_conf1 = qm_rdkit1.GetConformer()

### Geometries

In [14]:
atoms1=[a for a in opt1_sage200.GetAtoms()]

In [15]:
atom_indices_s1 = [a.GetIdx() for a in atoms1 if a.GetSymbol() == 'S']

In [16]:
atom_indices_n1 = [a.GetIdx() for a in atoms1 if a.GetSymbol()=='N']

In [17]:
atom_indices_o1 = [a.GetIdx() for a in atoms1 if a.GetSymbol()=='O']

In [18]:
print(atom_indices_s1)
print(atom_indices_n1)
print(atom_indices_o1)

[12]
[11, 13]
[9, 10, 14, 15]


In [19]:
s = atom_indices_s1[0]
n1,n2 = atom_indices_n1
o1,o2 = atom_indices_o1[2:]

In [20]:
Chem.rdMolTransforms.GetBondLength(opt1_sage200_conf,s,n1) # N

1.6836715526509654

In [21]:
Chem.rdMolTransforms.GetBondLength(opt1_sage200_conf,s,n2) # N

1.6124789163404611

In [22]:
Chem.rdMolTransforms.GetBondLength(opt1_sage200_conf,s,o1) # O

1.460868900132385

In [23]:
Chem.rdMolTransforms.GetBondLength(opt1_sage200_conf,s ,o2) # O

1.460405975073076

In [24]:
print('N1 - S - N2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage200_conf,n1,s,n2)) # N1 - S - N2
print('N1 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage200_conf,n1,s,o1)) # N1 - S - O1
print('N1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage200_conf,n1,s,o2)) # N1 - S - O2
print('O1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage200_conf,o1,s,o2)) # O1 - S - O2


N1 - S - N2: 63.44649302464608
N1 - S - O1: 113.26201724942982
N1 - S - O2: 111.27609044785295
O1 - S - O2: 116.04513687658701


In [25]:
print('N1 - S - N2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage210_conf,n1,s,n2)) # N1 - S - N2
print('N1 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage210_conf,n1,s,o1)) # N1 - S - O1
print('N1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage210_conf,n1,s,o2)) # N1 - S - O2
print('O1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage210_conf,o1,s,o2)) # O1 - S - O2


N1 - S - N2: 88.44799419407012
N1 - S - O1: 116.31464418178489
N1 - S - O2: 111.2756852853613
O1 - S - O2: 126.19997434184445


In [26]:
print('N1 - S - N2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage220_conf,n1,s,n2)) # N1 - S - N2
print('N1 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage220_conf,n1,s,o1)) # N1 - S - O1
print('N1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage220_conf,n1,s,o2)) # N1 - S - O2
print('O1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage220_conf,o1,s,o2)) # O1 - S - O2


N1 - S - N2: 98.65037944326272
N1 - S - O1: 110.82320280729996
N1 - S - O2: 106.289105189148
O1 - S - O2: 126.42764069419836


In [27]:
print('N1 - S - N2:',Chem.rdMolTransforms.GetAngleDeg(qm_conf1,n1,s,n2)) # N1 - S - N2
print('N1 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(qm_conf1,n1,s,o1)) # N1 - S - O1
print('N1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(qm_conf1,n1,s,o2)) # N1 - S - O2
print('O1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(qm_conf1,o1,s,o2)) # O1 - S - O2

N1 - S - N2: 99.05907369913429
N1 - S - O1: 115.56913768989678
N1 - S - O2: 107.47422746012008
O1 - S - O2: 118.59079770561428


# Molecule 31, QCArchive ID 36972425

Bad with Sage 2.0 and 2.1, fixed with 2.2

Has many conformers--index 29-38. Some are distorted, some aren't

In [119]:
test_mol2 = industry_benchmark_rm[31][1]
test_mol2

NGLWidget()

In [55]:
test_mol2.to_file('qcaid_36972425.sdf',file_format='sdf')

In [28]:
test_mol2 = Molecule.from_file('qcaid_36972425.sdf')

In [29]:
ie_sage200_qm2,me_sage200_qm,min_mol_sage200_qm2 = minimize_energy(test_mol2,ForceField('openff_unconstrained-2.0.0.offxml'))
min_mol_sage200_qm2

NGLWidget()

In [30]:
ie_sage210_qm2,me_sage210_qm2,min_mol_sage210_qm2 = minimize_energy(test_mol2,ForceField('openff_unconstrained-2.1.0.offxml'))
min_mol_sage210_qm2

NGLWidget()

In [32]:
ie_sage220_qm2,me_sage220_qm2,min_mol_sage220_qm2 = minimize_energy(test_mol2,ForceField('../../../openff_unconstrained-2.2.0-rc1.offxml',allow_cosmetic_attributes=True))
min_mol_sage220_qm2

NGLWidget()

In [33]:
min_mol_sage220_qm2.to_file('qcaid_36972425_optsage220.sdf',file_format='sdf')

In [34]:
opt2_sage200 = min_mol_sage200_qm2.to_rdkit()
opt2_sage200_conf = opt2_sage200.GetConformer()

In [35]:
opt2_sage210 = min_mol_sage210_qm2.to_rdkit()
opt2_sage210_conf = opt2_sage210.GetConformer()

In [36]:
opt2_sage220 = min_mol_sage220_qm2.to_rdkit()
opt2_sage220_conf = opt2_sage220.GetConformer()

In [37]:
qm_rdkit2 = test_mol2.to_rdkit()
qm_conf2 = qm_rdkit2.GetConformer()

### Geometries

In [38]:
atoms2=[a for a in opt2_sage200.GetAtoms()]

In [39]:
atom_indices_s2 = [a.GetIdx() for a in atoms2 if a.GetSymbol() == 'S']
atom_indices_n2 = [a.GetIdx() for a in atoms2 if a.GetSymbol()=='N']
atom_indices_o2 = [a.GetIdx() for a in atoms2 if a.GetSymbol()=='O']

In [40]:
print(atom_indices_s2)
print(atom_indices_n2)
print(atom_indices_o2)

[13]
[12, 16]
[4, 14, 15]


In [41]:
s = atom_indices_s2[0]
n1,n2 = atom_indices_n2
o1,o2 = atom_indices_o2[1:]

In [42]:
Chem.rdMolTransforms.GetBondLength(opt2_sage200_conf,s,n1) # N

1.6642934588330192

In [45]:
Chem.rdMolTransforms.GetBondLength(opt2_sage200_conf,s,n2) # N

1.6154552714032644

In [46]:
Chem.rdMolTransforms.GetBondLength(opt2_sage200_conf,s,o1) # O

1.459271963030652

In [47]:
Chem.rdMolTransforms.GetBondLength(opt2_sage200_conf,s ,o2) # O

1.4598227357268343

In [48]:
print('N1 - S - N2:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage200_conf,n1,s,n2)) # N1 - S - N2
print('N1 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage200_conf,n1,s,o1)) # N1 - S - O1
print('N2 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage200_conf,n2,s,o1)) # N1 - S - O1
print('N1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage200_conf,n1,s,o2)) # N1 - S - O2
print('O1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage200_conf,o1,s,o2)) # O1 - S - O2


N1 - S - N2: 72.11428788652712
N1 - S - O1: 115.44296488784543
N2 - S - O1: 79.33304874929252
N1 - S - O2: 114.32383441385863
O1 - S - O2: 115.9059387096902


In [49]:
print('N1 - S - N2:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage210_conf,n1,s,n2)) # N1 - S - N2
print('N1 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage210_conf,n1,s,o1)) # N1 - S - O1
print('N2 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage210_conf,n2,s,o1)) # N1 - S - O1
print('N1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage210_conf,n1,s,o2)) # N1 - S - O2
print('O1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage210_conf,o1,s,o2)) # O1 - S - O2



N1 - S - N2: 77.84322264570787
N1 - S - O1: 115.16684226994481
N2 - S - O1: 83.30668906742419
N1 - S - O2: 114.67684578569971
O1 - S - O2: 124.17870989745376


In [50]:
print('N1 - S - N2:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage220_conf,n1,s,n2)) # N1 - S - N2
print('N1 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage220_conf,n1,s,o1)) # N1 - S - O1
print('N2 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage220_conf,n2,s,o1)) # N1 - S - O1
print('N1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage220_conf,n1,s,o2)) # N1 - S - O2
print('O1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage220_conf,o1,s,o2)) # O1 - S - O2

N1 - S - N2: 103.41989396993706
N1 - S - O1: 108.42681683618302
N2 - S - O1: 102.61903295167431
N1 - S - O2: 110.49580662474334
O1 - S - O2: 126.08980272631219


In [51]:
print('N1 - S - N2:',Chem.rdMolTransforms.GetAngleDeg(qm_conf2,n1,s,n2)) # N1 - S - N2
print('N1 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(qm_conf2,n1,s,o1)) # N1 - S - O1
print('N2 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(qm_conf2,n2,s,o1)) # N1 - S - O1
print('N1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(qm_conf2,n1,s,o2)) # N1 - S - O2
print('O1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(qm_conf2,o1,s,o2)) # O1 - S - O2

N1 - S - N2: 110.18676461051349
N1 - S - O1: 105.13824314962861
N2 - S - O1: 105.5516174522408
N1 - S - O2: 106.29636623563432
O1 - S - O2: 124.68928935675294
